# RAG Tutorial

## สิ่งที่จะได้รู้
- การใช้งาน OpenAI Client
- การใช้งาน OpenAI API Compatibility
- การใช้งาน Role system, user, assistant
- การใช้งาน Embedding
- การใช้งาน VectorDB
- การสร้าง RAG

## สิ่งที่คาดหวัง
- สามารถสร้าง RAG โดยใช้ OpenAI API Compatibility ร่วมกับ Embedding ที่รองรับภาษาไทย

# การใช้งาน OpenAI Client

In [ ]:
!pip install rich

In [ ]:
from rich import print

In [ ]:
from openai import OpenAI # เรียกใช้งาน OpenAI Library

สำหรับการใช้งาน OpenAI Model โดยตรง แก้ไขเฉพาะ API Key เท่านั้น แต่ถ้าต้องการใช้งาน OpenAI API Compatibility จำเป็นต้องแก้ไข base_url ด้วย

In [ ]:
API_KEY="REDACTED_OPENAI_API_KEY" # ใช้งาน GPT-5 หรือ GPT ตัวอื่น ๆ
client = OpenAI(
    api_key = API_KEY,
)
base_model = "gpt-5.2"

In [ ]:
# API_KEY="float16-r-CT1EIdtNcJDOw015AAHj5XSlYKyn" # สำหรับใช้งานผ่าน Float16, Open router, Ollama หรือ Self-host ตัวอื่น ๆ
# client = OpenAI(
#     api_key = API_KEY,
#     base_url="https://proxy-instance.float16.cloud/7188aae3-1e71-4d7c-a6ae-f50ce9cf1983/3900/v1"
# )
# model = "/model/Qwen/Qwen3-VL-30B-A3B-Instruct-FP8"

`chat.completion.create` คือการเรียกใช้งาน model โดยมี arguments สำคัญได้แก่
1. `model` ใช้สำหรับระบุว่าต้องการใช้งาน model ตัวไหน
2. `messages` ใช้สำหรับระบุ prompt และการสนทนาแบบ multi-turn
3. `max_tokens` ใช้สำหรับระบุความยาวสูงสุดสำหรับการตอบกลับ
4. `stream` ใช้ระบุว่า Response ควรตอบกลับมาเป็น Streaming หรือ Non-Streaming

In [ ]:
res = client.chat.completions.create(
    model=base_model,
    messages=[
        {
            "role": "user",
            "content": "สวัสดี คุณทำอะไรได้บ้าง"
        },
    ],
    stream=False
)
print(res)

ChatCompletion(
    id='chatcmpl-D7dhRNLZpAo8aYtAsSuJhKqXRmcX1',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='ผมช่วยได้หลายอย่าง ขึ้นกับสิ่งที่คุณต้องการ เช่น\n\n- ตอบคำถาม/อธิบายความรู้ (วิทย์ คณิต ประวัติ เทคโนโลยี
ฯลฯ)\n- ช่วยเขียนและปรับแก้ข้อความ: อีเมล รายงาน เรซูเม่ โพสต์ โครงร่างบทความ\n- แปลภาษาและช่วยปรับสำนวน (ไทย–อังกฤษ และภาษาอื่น
ๆ)\n- สรุปเอกสาร/บทความ และดึงประเด็นสำคัญ\n- ช่วยคิดไอเดีย วางแผนงาน การเรียน การท่องเที่ยว เมนูอาหาร\n- ช่วยเขียนโค้ด อธิบายโค้ด
ดีบัก และออกแบบอัลกอริทึม\n- วิเคราะห์รูปภาพที่คุณส่งมา (เช่น อ่านข้อมูลจากภาพ อธิบายสิ่งที่เห็น)\n\nอยากให้ช่วยเรื่องไหนเป็นพิเศษ
บอกเป้าหมายและรายละเอียดสั้น ๆ ได้เลยครับ/ค่ะ เช่น “ช่วยเขียนอีเมลขอลางาน” หรือ “สรุปบทความนี้”',
                refusal=None,
                role='assistant',
                annotations=[],
                audio=None,
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1770713273,
    model='gpt-5.2-2025-12-11',
    object='chat.completion',
    service_tier='default',
    system_fingerprint=None,
    usage=CompletionUsage(
        completion_tokens=252,
        prompt_tokens=16,
        total_tokens=268,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

`messages` มีโครงสร้างและสำคัญต่อการเรียกใช้งาน LLM

`messages` มี data structure เป็น list of dict

โดยลำดับของ dictionary มีความสำคัญต่อ LLM เช่นกัน
โดยทั่วไปแล้วลำดับของ `messages` จะประกอบไปด้วย role ดังนี้

1. role : system (Optional)
2. role : user
3. role : assistant
4. role : user
5. role : assistant

ตามลำดับไปเรื่อย ๆ

ซึ่งแต่ละ role มีความสำคัญดังนี้

1. role : system

role : system ใช้สำหรับกำหนดคุณลักษณะของ LLM ที่เราต้องการให้ทำงานตามที่เราต้องการและใช้สำหรับการกำหนด กฎ, สูตร หรือแม้แต่ตัวย่อ ต่าง ๆ

role : user ใช้สำหรับ user prompt เมื่อ user ถามใน Chat Application ทุกครั้งจะถูกนำมาใส่ใน role : user เสมอ

role : assistant ใช้สำหรับเพิ่มสิ่งที่ LLM ได้ทำการตอบกลับมาจาก Response ครั้งล่าสุด

In [ ]:
messages = [
        {
            "role": "user",
            "content": "สวัสดี คุณคือสุดยอดผู้ช่วยด้านฝ่ายขาย ชื่อ Mati"
        }
]

In [ ]:
res = client.chat.completions.create(
    model=base_model,
    messages=messages,
    stream=False
)
print(res)

ChatCompletion(
    id='chatcmpl-D7dhWyQfUTJKm5LlKWoftb67rvHE3',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='สวัสดีครับ ผม Mati ผู้ช่วยด้านฝ่ายขายของคุณครับ  \nบอกผมได้เลยว่าคุณขายสินค้า/บริการอะไร 
กลุ่มลูกค้าเป้าหมายคือใคร และอยากให้ผมช่วยเรื่องไหนเป็นพิเศษ เช่น\n\n- เขียนสคริปต์ทักแชท/ปิดการขาย  \n- ทำข้อเสนอ (Proposal) /
ใบเสนอราคา  \n- วางโครงสร้าง Sales funnel และแผนติดตามลูกค้า  \n- จัดการข้อโต้แย้ง/ตอบคำถามลูกค้า  \n- 
ทำข้อความโฆษณา/คอนเทนต์เพื่อปิดการขาย\n\nเริ่มจาก: สินค้าหรือบริการของคุณคืออะไร และขายผ่านช่องทางไหนครับ?',
                refusal=None,
                role='assistant',
                annotations=[],
                audio=None,
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1770713278,
    model='gpt-5.2-2025-12-11',
    object='chat.completion',
    service_tier='default',
    system_fingerprint=None,
    usage=CompletionUsage(
        completion_tokens=165,
        prompt_tokens=21,
        total_tokens=186,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

In [ ]:
assistant_response = res.choices[0].message.content

In [ ]:
messages.append({ #เพิ่มบทสนทนาล่าสุดจาก LLM ลงไปในการสนทนา
    "role" : "assistant",
    "content" : assistant_response
})

In [ ]:
messages.append({ #เพิ่มบทสนทนาล่าสุดจาก LLM ลงไปในการสนทนา
    "role" : "user",
    "content" : "สวัสดี คุณทำอะไรได้บ้าง"
})

In [ ]:
print(messages)

[
    {'role': 'user', 'content': 'สวัสดี คุณคือสุดยอดผู้ช่วยด้านฝ่ายขาย ชื่อ Mati'},
    {
        'role': 'assistant',
        'content': 'สวัสดีครับ ผม Mati ผู้ช่วยด้านฝ่ายขายของคุณครับ  \nบอกผมได้เลยว่าคุณขายสินค้า/บริการอะไร กลุ่มลูกค้าเป้าหมายคือใคร
และอยากให้ผมช่วยเรื่องไหนเป็นพิเศษ เช่น\n\n- เขียนสคริปต์ทักแชท/ปิดการขาย  \n- ทำข้อเสนอ (Proposal) / ใบเสนอราคา  \n-
วางโครงสร้าง Sales funnel และแผนติดตามลูกค้า  \n- จัดการข้อโต้แย้ง/ตอบคำถามลูกค้า  \n- 
ทำข้อความโฆษณา/คอนเทนต์เพื่อปิดการขาย\n\nเริ่มจาก: สินค้าหรือบริการของคุณคืออะไร และขายผ่านช่องทางไหนครับ?'
    },
    {'role': 'user', 'content': 'สวัสดี คุณทำอะไรได้บ้าง'}
]

## ลักษณะของ messages สำหรับ Multi-turn

ลักษณะของ `messages` ประกอบไปด้วย

1. role : user
2. role : assistant
3. role : user

โดย role : assistant เป็นการนำคำตอบจาก LLM ครั้งก่อนมาใส่ไว้ เพื่อให้ LLM สามารถจดจำการสนทนาครั้งก่อนได้

In [ ]:
next_turn_response = client.chat.completions.create(
    model=base_model,
    messages=messages,
    stream=False
)
assistant_response = next_turn_response.choices[0].message.content

In [ ]:
print(assistant_response)

ผม Mati ผู้ช่วยด้าน “ฝ่ายขาย” ช่วยได้ครบตั้งแต่หาลูกค้าไปจนถึงปิดการขายและดูแลหลังการขายครับ เช่น

1) วางกลยุทธ์และแผนการขาย  
- กำหนดกลุ่มเป้าหมาย (ICP), จุดขาย (Value Proposition), แพ็กเกจ/ราคา  
- วาง Sales funnel และ KPI รายสัปดาห์/รายเดือน

2) เขียนสคริปต์ขายทุกช่องทาง  
- สคริปต์ทักแชท/ไลน์/DM, โทรขาย, ตอบคอมเมนต์  
- สคริปต์นัดเดโม/นัดคุย และสคริปต์ปิดการขาย

3) ทำข้อความ/คอนเทนต์เพื่อยอดขาย  
- โพสต์ขาย, โฆษณา, Landing page, ข้อความ Broadcast  
- ปรับให้เหมาะกับสินค้าและกลุ่มลูกค้า

4) จัดการข้อโต้แย้ง (Objection handling)  
- แพงไป/ยังไม่พร้อม/ขอคิดดูก่อน/เทียบเจ้าอื่น/ไม่มั่นใจผลลัพธ์  
- ทำชุดคำตอบแบบสุภาพแต่พาลูกค้าไปต่อ

5) ทำเอกสารขาย  
- Proposal, ใบเสนอราคา, Pitch deck, FAQ, One-pager  
- โครงสร้างข้อเสนอที่โน้มน้าวและอ่านง่าย

6) ระบบติดตามลูกค้าและ CRM  
- ทำขั้นตอน Follow-up, ข้อความติดตาม, ตั้งสถานะลีด  
- ออกแบบเทมเพลตใช้งานใน Google Sheet/CRM

ถ้าบอก 3 อย่างนี้ ผมจะช่วยจัดให้เป็นชุดพร้อมใช้ได้เลย:  
1) คุณขายอะไร (สินค้า/บริการ)  
2) ลูกค้าเป้าหมายคือใคร  
3) ขายผ่านช่องทางไหน (แชท/โทร/หน้าร้าน/ออนไลน์)

In [ ]:
single_turn_response = client.chat.completions.create(
    model=base_model,
    messages=[{
        "role" : "user",
        "content" : "สวัสดี คุณทำอะไรได้บ้าง"
    }],
    stream=False
)
print(single_turn_response.choices[0].message.content)

ผมช่วยได้หลายอย่าง ขึ้นอยู่กับสิ่งที่คุณต้องการ เช่น

- ตอบคำถาม/อธิบายความรู้: วิทย์ คณิต ประวัติศาสตร์ เศรษฐศาสตร์ ไอที ฯลฯ  
- แปลภาษาและช่วยเขียน: ไทย–อังกฤษ–ภาษาอื่น ๆ, ปรับสำนวน ตรวจแกรมมาร์ สรุป/เรียบเรียง  
- ช่วยคิดไอเดีย: คอนเทนต์ โฆษณา ชื่อแบรนด์ แผนงาน กลยุทธ์  
- สรุปเอกสาร/บทความ: จับประเด็น ทำ bullet point ทำโน้ต  
- เขียนโค้ด/ดีบัก: Python, JavaScript, SQL ฯลฯ พร้อมอธิบาย  
- วางแผนการเรียน/การทำงาน: ตารางอ่านหนังสือ แผนโปรเจกต์ To-do  
- ช่วยวิเคราะห์ข้อมูลเบื้องต้น: แนวทางทำรายงาน/กราฟ/การตีความ  
- ช่วยจากรูปภาพ: อธิบายภาพ อ่านข้อมูลจากภาพ (ถ้ามี)

ตอนนี้คุณอยากให้ช่วยเรื่องอะไรเป็นพิเศษครับ/คะ?

## เปรียบเทียบการทำ Single-turn และ Multi-turn

จากการทดสอบระหว่างการเพิ่ม assistant และไม่เพิ่ม assistant จะเห็นได้ว่า
เมื่อไม่เพิ่ม assistant ลงไปใน `messages` LLM จะไม่สามารถจดจำการสนทนาครั้งก่อนได้เลย

ทำให้ **LLM ทำงานเป็น stateless เท่านั้น** ไม่มีการเก็บค่าการสนทนาครั้งก่อนไว้กับตัวเอง

## Embedding

Embedding เป็น AI Model สำหรับการแปลง Text ให้เป็น Vector เพื่อที่จะสามารถทำ Semantic search ได้

โดยเราจะใช้ Embedding ชื่อ bge-m3 ซึ่งรองรับภาษาไทย และ all-MiniLM-L6-v2 ซึ่งไม่รองรับภาษาไทย เพื่อเปรียบเทียบประสิทธิภาพ

ติดตั้ง bge-m3 ผ่าน pip install

In [ ]:
! pip install -U FlagEmbedding==1.3.5

In [ ]:
! pip install transformers==4.57.6

In [ ]:
from FlagEmbedding import BGEM3FlagModel
import numpy as np


ทดสอบการหาความใกล้เคียงระหว่างประโยคภาษาไทยด้วย BGE-M3 (ค่ายิ่งมากยิ่งใกล้เคียง)

In [ ]:
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

sentences_1 = ["ไก่ถูกใช้ทำอาหารอะไรบ้าง"] # ทดสอบด้วยภาษาไทย
sentences_2 = ["กะเพราหมู", "ข้าวมันไก่", "ก๋วยเตี๋ยว"]
sentences_3 = ["กะเพราหมู", "ข้าวมันไก่", "ก๋วยเตี๋ยว","กะเพราหมู", "ข้าวมันไก่", "ก๋วยเตี๋ยว"]

embeddings_1 = model.encode(sentences_1,
                            batch_size=12,
                            max_length=8192,
                            )['dense_vecs']
embeddings_2 = model.encode(sentences_2)['dense_vecs']
print(embeddings_1)
print(embeddings_2)
print(len(embeddings_2))
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[[ 0.02559   0.01767  -0.02658  ...  0.002422 -0.01714   0.007423]]

[[-0.027    -0.02573  -0.01185  ...  0.04108  -0.04965   0.01727 ]
 [ 0.03406   0.04202  -0.0503   ...  0.014305 -0.06396   0.002642]
 [-0.04074   0.06015  -0.01107  ...  0.01236  -0.01984  -0.01831 ]]

3

[[0.4075 0.553  0.4087]]

ทดสอบการหาความใกล้เคียงระหว่างประโยคภาษาไทยด้วย all-MiniLM-L6-v2 (ค่ายิ่งมากยิ่งใกล้เคียง)

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["ไก่ถูกใช้ทำอาหารอะไรบ้าง", "กะเพราหมู", "ข้าวมันไก่", "ก๋วยเตี๋ยว"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
len(embeddings)

4

In [ ]:
similarity = embeddings[0] @ embeddings[1:].T
print(similarity)

[0.99999994 0.99999994 0.20779707]

In [ ]:
# @title def store_content_with_vector
from typing import List, Dict, Any, Tuple, Optional
import math

collection = {}

def store_content_with_vector(content_and_vector, collection_key):
    """
    Store content and vector data under a specific key in the collection.

    Args:
        content_and_vector: The data to store. Expected structure:
            - Can be any object (dict, list, tuple, string, etc.)
            - Commonly used as a dict with keys like:
                - "content": (str, list, dict) - the actual content
                - "vector": (list, tuple, np.ndarray) - numerical vector representation
                - "metadata": (dict, optional) - additional info (e.g., source, timestamp)
            Example:
                {
                    "content": "This is a document",
                    "vector": [0.1, 0.2, 0.3]
                }
            - Can also be a simple string, list, or any serializable object.

        collection_key (str): The key under which to store the data. Must be a string.

    Returns:
        dict: Result containing success status and message
    """
    try:
        if collection_key in collection:
            # Append content_and_vector to existing key
            collection[collection_key].append(content_and_vector)
            return {
                "success": True,
                "message": f"Content and vector successfully appended to existing key '{collection_key}'"
            }
        else:
            # Create new key with content_and_vector as a list
            collection[collection_key] = [content_and_vector]
            return {
                "success": True,
                "message": f"New key '{collection_key}' created and content/vector stored"
            }
    except Exception as e:
        return {
            "success": False,
            "error": f"Unexpected error occurred during storage: {str(e)}"
        }

def clear_collection():
  global collection
  collection = {}

In [ ]:
# @title def get_content
def get_content(
    query_vector: List[float],
    top_k: int = 5,
    threshold: float = 0.0,
    collection_key: str = None,
    return_vector = False
) -> List[Dict[str, Any]]:
    global collection
    """
    Find the most similar content entries to a query vector using cosine similarity.

    Args:
        query_vector (List[float]): The query vector to search for (must be non-empty)
        top_k (int): Maximum number of results to return (default: 5)
        threshold (float): Minimum similarity score threshold (0.0 to 1.0, default: 0.0)
        collection_key (str, optional): Specific key in collection to search. If None, searches all keys.

    Returns:
        List[Dict[str, Any]]: List of matching results with:
            - "content": The stored content
            - "vector": The stored vector
            - "similarity": Cosine similarity score (0.0 to 1.0)
            - "metadata": Any additional metadata (if present)
            - "collection_key": The key where this entry was found

    Raises:
        ValueError: If query_vector is empty or invalid
        TypeError: If parameters have wrong types
        RuntimeError: If no data is available for searching
    """
    try:
        # Input validation
        if not isinstance(query_vector, (list, tuple)):
            return {
                "success": False,
                "error": f"query_vector must be a list or tuple, got {type(query_vector).__name__}"
            }

        if not query_vector:
            return {
                "success": False,
                "error": "query_vector cannot be empty"
            }

        # Check if collection exists and has data
        if not collection:
            return {
                "success": False,
                "error": "Collection is empty. No data to search."
            }

        # Determine which keys to search
        keys_to_search = [collection_key] if collection_key else collection.keys()

        # Validate that the specified key exists if provided
        if collection_key and collection_key not in collection:
            return {
                "success": False,
                "error": f"collection_key '{collection_key}' does not exist in the collection"
            }

        # List to store results
        results = []
        # Search through all relevant keys
        for key in keys_to_search:
            if not isinstance(collection[key], list):
                continue  # Skip non-list entries

            for item in collection[key]:
                # Extract vector from item
                if isinstance(item, dict):
                    vector_data = item.get("vector")
                    content = item.get("content", item)  # Use content field or the item itself
                    metadata = item.get("metadata", {})
                elif isinstance(item, (list, tuple)):
                    vector_data = item
                    content = item
                    metadata = {}
                else:
                    # Handle other types (string, number, etc.)
                    vector_data = None
                    content = item
                    metadata = {}

                # Skip if no vector data available
                if vector_data is None or not isinstance(vector_data, (list, tuple)):
                    continue

                # Ensure vector dimensions match
                if len(query_vector) != len(vector_data):
                    continue

                # Calculate cosine similarity
                try:
                    # Calculate dot product
                    dot_product = sum(a * b for a, b in zip(query_vector, vector_data))

                    # Calculate magnitudes
                    query_magnitude = math.sqrt(sum(a * a for a in query_vector))
                    vector_magnitude = math.sqrt(sum(b * b for b in vector_data))

                    # Avoid division by zero
                    if query_magnitude == 0 or vector_magnitude == 0:
                        similarity = 0.0
                    else:
                        similarity = dot_product / (query_magnitude * vector_magnitude)

                    # Apply threshold
                    if similarity < threshold:
                        continue


                    if return_vector :
                      # Add to results
                      results.append({
                          "content": content,
                          "vector": vector_data,
                          "similarity": round(similarity, 6),
                          "metadata": metadata,
                          "collection_key": key
                      })
                    else :
                      results.append({
                          "content": content,
                          "similarity": round(similarity, 6),
                          "metadata": metadata,
                          "collection_key": key
                      })

                except (TypeError, ValueError, OverflowError) as e:
                    # Skip items with calculation errors
                    continue

        # Sort by similarity (descending) and take top_k
        results.sort(key=lambda x: x["similarity"], reverse=True)
        top_results = results[:top_k]

        # Return results
        return {
            "success": True,
            "results": top_results,
            "total_found": len(results),
            "top_k": top_k,
            "threshold": threshold
        }

    except Exception as e:
        return {
            "success": False,
            "error": f"Unexpected error occurred during search: {str(e)}"
        }


In [ ]:
# store_content_with_vector(content_and_vector, collection_key)

store_content_with_vector({
  "content": "This is a document",
  "vector": [0.1, 0.2, 0.3]
}, 'test')

{'success': True,
 'message': "New key 'test' created and content/vector stored"}

In [ ]:
#get_content(query_vector = List[Float], top_k: int = 5, threshold: float = 0.0, collection_key: str = None )

get_content(query_vector = [0.1,0.2,0.3], collection_key = "test", return_vector = True)

{'success': True,
 'results': [{'content': 'This is a document',
   'vector': [0.1, 0.2, 0.3],
   'similarity': 1.0,
   'metadata': {},
   'collection_key': 'test'}],
 'total_found': 1,
 'top_k': 5,
 'threshold': 0.0}

In [ ]:
documents = [
    "ข้าวมันไก่ - ทำจากข้าวสวยต้มในน้ำซุปไก่ พร้อมไก่ต้มน้ำซุป น้ำจิ้ม",
    "กะเพรา - ทำจากข้าวสวยราดด้วยกับกะเพรา ไก่หรือหมู พร้อมน้ำปลา น้ำตาล และพริกขี้หนู",
    "ก๋วยเตี๋ยว - ทำจากเส้นก๋วยเตี๋ยวเหนียวนุ่ม ผัดหรือต้มในน้ำซุปไก่หรือเนื้อ ใส่เนื้อสัตว์ ผัก และเครื่องปรุงรส",
    "ผัดไท - ทำจากเส้นก๋วยเตี๋ยวเหนียว ผัดกับไข่ ถั่วงอก และน้ำปลา น้ำมะนาว น้ำตาล"
]
def do_vector_by_yourself(documents):

  model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
  for doc in documents :
      embedded_result = model.encode(doc,
                                  batch_size=1,
                                  max_length=8192,
                                  )['dense_vecs']
      store_content_with_vector({
          "content" : doc,
          "vector" : list(embedded_result)
      },'my_collection')

do_vector_by_yourself(documents)

## Search

user_prompt = "แนะนำเมนูที่ทำมาจากเส้นหน่อย"

def do_embedding_by_yourself(user_prompt):
  user_query = None
  model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
  user_query = model.encode(user_prompt,
                              batch_size=1,
                              max_length=8192,
                              )['dense_vecs']
  return user_query

query_vector = do_embedding_by_yourself(user_prompt)

print(query_vector, type(query_vector), query_vector.shape)
search_result = get_content(query_vector = list(query_vector), collection_key = "my_collection")
clear_collection()
print(search_result)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 1590.56it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00, 59.14it/s]


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 1846.08it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00, 46.26it/s]


[-0.03102   0.03128  -0.04706  ... -0.00384   0.00364  -0.000893] <class 'numpy.ndarray'>
(1024,)

/tmp/ipython-input-3880990959.py:126: RuntimeWarning: overflow encountered in cast
  "similarity": round(similarity, 6),
/tmp/ipython-input-3880990959.py:126: RuntimeWarning: invalid value encountered in divide
  "similarity": round(similarity, 6),


{
    'success': True,
    'results': [
        {
            'content': 'ข้าวมันไก่ - ทำจากข้าวสวยต้มในน้ำซุปไก่ พร้อมไก่ต้มน้ำซุป น้ำจิ้ม',
            'similarity': np.float16(nan),
            'metadata': {},
            'collection_key': 'my_collection'
        },
        {
            'content': 'กะเพรา - ทำจากข้าวสวยราดด้วยกับกะเพรา ไก่หรือหมู พร้อมน้ำปลา น้ำตาล และพริกขี้หนู',
            'similarity': np.float16(nan),
            'metadata': {},
            'collection_key': 'my_collection'
        },
        {
            'content': 'ก๋วยเตี๋ยว - ทำจากเส้นก๋วยเตี๋ยวเหนียวนุ่ม ผัดหรือต้มในน้ำซุปไก่หรือเนื้อ ใส่เนื้อสัตว์ ผัก และเครื่องปรุงรส',
            'similarity': np.float16(nan),
            'metadata': {},
            'collection_key': 'my_collection'
        },
        {
            'content': 'ผัดไท - ทำจากเส้นก๋วยเตี๋ยวเหนียว ผัดกับไข่ ถั่วงอก และน้ำปลา น้ำมะนาว น้ำตาล',
            'similarity': np.float16(nan),
            'metadata': {},
            'collection_key': 'my_collection'
        }
    ],
    'total_found': 4,
    'top_k': 5,
    'threshold': 0.0
}

## นำผลลัพธ์จาก Vector Database ใช้ร่วมกับ LLM model

In [ ]:
retrieved_content = ""
for item in search_result["results"] :
    content = item['content']
    retrieved_content += content + "\n"

retrieved_content

'ข้าวมันไก่ - ทำจากข้าวสวยต้มในน้ำซุปไก่ พร้อมไก่ต้มน้ำซุป น้ำจิ้ม\nกะเพรา - ทำจากข้าวสวยราดด้วยกับกะเพรา ไก่หรือหมู พร้อมน้ำปลา น้ำตาล และพริกขี้หนู\nก๋วยเตี๋ยว - ทำจากเส้นก๋วยเตี๋ยวเหนียวนุ่ม ผัดหรือต้มในน้ำซุปไก่หรือเนื้อ ใส่เนื้อสัตว์ ผัก และเครื่องปรุงรส\nผัดไท - ทำจากเส้นก๋วยเตี๋ยวเหนียว ผัดกับไข่ ถั่วงอก และน้ำปลา น้ำมะนาว น้ำตาล\n'

In [ ]:
user_prompt = "แนะนำเมนูที่ทำมาจากเส้นหน่อย"
messages = [
        {
            "role": "user",
            "content": f"""
            ===========================CONTEXT===========================
            Context : {retrieved_content}
            ===========================CONTEXT===========================

            User question : {user_prompt}
            """
        }
]
print(messages)

[
    {
        'role': 'user',
        'content': '\n            ===========================CONTEXT===========================\n            
Context : ข้าวมันไก่ - ทำจากข้าวสวยต้มในน้ำซุปไก่ พร้อมไก่ต้มน้ำซุป น้ำจิ้ม\nกะเพรา - ทำจากข้าวสวยราดด้วยกับกะเพรา ไก่หรือหมู พร้อมน้ำปลา
น้ำตาล และพริกขี้หนู\nก๋วยเตี๋ยว - ทำจากเส้นก๋วยเตี๋ยวเหนียวนุ่ม ผัดหรือต้มในน้ำซุปไก่หรือเนื้อ ใส่เนื้อสัตว์ ผัก และเครื่องปรุงรส\nผัดไท -
ทำจากเส้นก๋วยเตี๋ยวเหนียว ผัดกับไข่ ถั่วงอก และน้ำปลา น้ำมะนาว น้ำตาล\n\n            
===========================CONTEXT===========================\n\n            User question : 
แนะนำเมนูที่ทำมาจากเส้นหน่อย\n            '
    }
]

In [ ]:
response = client.chat.completions.create(
    model=base_model,
    messages=[{"role":"user", "content" : user_prompt}],
    stream=False
)
print(response)

ChatCompletion(
    id='chatcmpl-D7diwm9Sa8vYO9Asw9YRStPjvrpYR',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='ได้เลย นี่คือเมนู “ทำจากเส้น” ที่ทำได้หลากหลายแบบ แบ่งตามสไตล์ให้เลือกง่าย ๆ\n\n**เมนูเส้นไทย**\n-
ผัดไทยกุ้งสด / ผัดไทยเจ  \n- ผัดซีอิ๊วหมู/ไก่/ทะเล  \n- ราดหน้าหมี่กรอบ / ราดหน้าเส้นใหญ่  \n- ก๋วยเตี๋ยวคั่วไก่  \n- ก๋วยเตี๋ยวเรือ  \n-
เย็นตาโฟ  \n- ขนมจีนแกงเขียวหวาน / น้ำยา / ซาวน้ำ  \n- สุกี้น้ำ/แห้ง (วุ้นเส้น)\n\n**เมนูเส้นสไตล์เกาหลี**\n- จับแช (วุ้นเส้นผัดเกาหลี)
\n- จาจังมยอน (บะหมี่ซอสดำ)  \n- รามยอนผัด (볶음라면) ใส่ไข่/ชีส\n\n**เมนูเส้นสไตล์ญี่ปุ่น**\n- ยากิโซบะ (ผัดโซบะ)  \n- อุด้งผัดซอส /
อุด้งน้ำ  \n- โซเมนเย็น (กินกับน้ำจิ้มสึยุ)\n\n**เมนูเส้นฝรั่ง/อิตาเลียน**\n- สปาเกตตี้ผัดขี้เมา  \n- คาโบนารา  \n- อะกลิโอ เอ โอลิโอ
(กระเทียม+น้ำมันมะกอก)  \n- โบโลเนส  \n- มักกะโรนีผัดซอสมะเขือเทศ\n\nถ้าบอกเพิ่มได้ว่า **มีเส้นอะไรอยู่ 
(เส้นเล็ก/เส้นใหญ่/บะหมี่/วุ้นเส้น/มักกะโรนี/สปาเกตตี้)** และอยากได้ **ผัดหรือแบบน้ำ** เดี๋ยวช่วยเลือก 3–5 เมนูที่เหมาะสุด
พร้อมวิธีทำแบบย่อให้ได้เลย.',
                refusal=None,
                role='assistant',
                annotations=[],
                audio=None,
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1770713366,
    model='gpt-5.2-2025-12-11',
    object='chat.completion',
    service_tier='default',
    system_fingerprint=None,
    usage=CompletionUsage(
        completion_tokens=471,
        prompt_tokens=20,
        total_tokens=491,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

In [ ]:
response = client.chat.completions.create(
    model=base_model,
    messages=messages,
    stream=False
)
print(response)

ChatCompletion(
    id='chatcmpl-D7dj93qot0meAhY6NKTO6veo9xDR9',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='เมนูที่ทำมาจาก “เส้น” จากตัวเลือกในบริบท มี 2 เมนูนี้:\n\n- **ก๋วยเตี๋ยว** — ทำจากเส้นก๋วยเตี๋ยวเหนียวนุ่ม
ผัดหรือต้มในน้ำซุปไก่หรือเนื้อ ใส่เนื้อสัตว์ ผัก และเครื่องปรุงรส  \n- **ผัดไท** — ทำจากเส้นก๋วยเตี๋ยวเหนียว ผัดกับไข่ ถั่วงอก และปรุงด้วยน้ำปลา
น้ำมะนาว น้ำตาล',
                refusal=None,
                role='assistant',
                annotations=[],
                audio=None,
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1770713379,
    model='gpt-5.2-2025-12-11',
    object='chat.completion',
    service_tier='default',
    system_fingerprint=None,
    usage=CompletionUsage(
        completion_tokens=135,
        prompt_tokens=220,
        total_tokens=355,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)